# Cluster/local configs

In [ ]:
# If on cluster, cwd is '/rds/general/user/nz423', not the project directory, breaking imports
import os
import sys
from pathlib import Path

project_dir = Path(os.getcwd()) / "exploring-hydra-boosting"
if "rds" not in project_dir.parts:
    project_dir = project_dir.parent
    datasets_dir = project_dir.parent.parent / "Data"
else:
    datasets_dir = project_dir / "DATASETS"
    if str(project_dir) not in sys.path:
        sys.path.append(str(project_dir))
    
class Config:
    project_dir = project_dir
    TSER_dir = datasets_dir / "TSER"
    BigTSC_dir = datasets_dir / "BigTSC"
    UTSC_dir = datasets_dir / "UTSC"

print(Config.project_dir)
print(Config.TSER_dir)
print(Config.BigTSC_dir)
print(Config.UTSC_dir)

# Code for running test

# Simple Wrapper no gridsearch

In [ ]:
# class TSMLWrapperHydraBoost(RegressorMixin, BaseTimeSeriesEstimator):
    
#     def __init__(self, **kwargs):
#         super(TSMLWrapperHydraBoost, self).__init__()
#         self.hydraboost = HydraBoost(
#             n_layers=1,
#             init_n_kernels=8,
#             init_n_groups=64,
#             n_kernels=8,
#             n_groups=64,
#             max_num_channels=3,
#             hydra_batch_size=10000,
#             l2_reg=10,
#             l2_ghat=0.1,
#             boost_lr=1,
#             train_top_at = [0, 5, 10],
#             **kwargs
#         )
        

#     def fit(self, X: np.ndarray, y: np.ndarray) -> object:
#         """Fit the estimator to training data.

#         Parameters
#         ----------
#         X : 3D np.ndarray of shape (n_instances, n_channels, n_timepoints)
#             The training data.
#         y : 1D np.ndarray of shape (n_instances)
#             The target labels for fitting, indices correspond to instance indices in X

#         Returns
#         -------
#         self :
#             Reference to self.
#         """
#         X = torch.from_numpy(X).float()
#         y = torch.from_numpy(y).float()
#         y = y.unsqueeze(1)
#         self.X_mean = X.mean()
#         self.X_std = X.std()
#         self.y_mean = y.mean()
#         self.y_std = y.std()
#         X = (X - self.X_mean) / self.X_std
#         y = (y - self.y_mean) / self.y_std
#         self.hydraboost.fit(X, y)
#         return self


#     def predict(self, X: np.ndarray) -> np.ndarray:
#         """Predicts labels for sequences in X.

#         Parameters
#         ----------
#         X : 3D np.ndarray of shape (n_instances, n_channels, n_timepoints)
#             The training data.

#         Returns
#         -------
#         y : array-like of shape (n_instances)
#             Predicted target labels.
#         """
#         X = torch.from_numpy(X).float()
#         X = (X - self.X_mean) / self.X_std
#         pred = self.hydraboost(X)
#         pred = pred * self.y_std + self.y_mean
#         return pred.squeeze().detach().numpy()
        
        

#     def _more_tags(self) -> dict:
#         return {
#             "X_types": ["3darray"],
#             "equal_length_only": True,
#             "allow_nan": False,
#         }

In [ ]:
# test_regressor(
#     regressor = TSMLWrapperHydraBoost(),
#     regressor_name = "HydraBoost",
# )

# Gridsearch Wrapper

In [ ]:
from run_regression_gridsearch import test_regressor, TSMLGridSearchWrapper
from models.random_feature_representation_boosting import HydraBoost


test_regressor(
    regressor_name = "HydraBoostGridSearchHoldout",
    regressor = TSMLGridSearchWrapper(
                "holdout",
                seed=0,
                device="cpu",
                modelClass=HydraBoost,
                model_param_grid={
                    "n_layers": [1],          
                    "init_n_kernels": [8],
                    "init_n_groups": [64],
                    "n_kernels": [8],
                    "n_groups": [64],
                    "max_num_channels": [3],
                    "hydra_batch_size": [10000],
                    "l2_reg": [0.01],               
                    "l2_ghat": [0.01],         
                    "boost_lr": [0.5],
                },
                ),
    dataset_name = "HouseholdPowerConsumption1",
    TSER_data_dir = Config.TSER_dir,
    results_dir = Config.project_dir / "results",
    resample_id=0,
)


In [ ]:
# !python run_regression_gridsearch.py \
#     --dataset_indices 0 \
#     --resample_ids 0 \
#     --results_dir "C:\\Users\\nz423\\Code\\exploring-hydra-boosting\\results" \
#     --TSER_dir "C:\\Users\\nz423\\Data\\TSER" \
#     --device "cpu" \
#     --seed 0